In [12]:
from pyspark import SparkContext, SparkConf

In [13]:
from operator import add

In [14]:
conf = SparkConf().setMaster("local[*]")
sc = SparkContext.getOrCreate()

In [15]:
def rdd_to_dictionary(rdd):
    return sc.parallelize(rdd).collectAsMap()

In [16]:
def write_json(dictionary,filename):
    import json  

    with open(filename, "w") as outfile:  
        json.dump(dictionary, outfile)    

In [17]:
documents = ["data/4300-0.txt","data/pg19033.txt", "data/pg36.txt","data/pg514.txt","data/pg1497.txt","data/pg3207.txt","data/pg42671.txt","data/pg6130.txt"]
data_bag = spark.read.text(documents).rdd.map(lambda r: r[0])

In [18]:
words_count = data_bag.flatMap(lambda x: x.split(' '))\
                .map(lambda x: x.lower()) \
                .map(lambda x: (x, 1))\
                .reduceByKey(add)\
                .filter(lambda x: x[1] > 1)\
                .filter(lambda x: len(x[0]) > 0)

In [19]:
stop_words_data_bag = spark.read.text("data/stopwords.txt").rdd.map(lambda r: r[0])
stop_words = stop_words_data_bag.flatMap(lambda x: x.split('\n'))\
                .map(lambda x: x.lower()).map(lambda x: (x, 1)).reduceByKey(add)
# print(stop_words.collect())

In [21]:
words_without_stop_words = words_count.subtractByKey(stop_words)
top_40_words_without_stop_words = words_without_stop_words.takeOrdered(40, key = lambda x: -x[1])

# print(top_40_words_without_stop_words)

In [22]:
top_40_words_without_stop_words_dictionary = rdd_to_dictionary(top_40_words_without_stop_words)
# print(top_40_words_without_stop_words_dictionary)

{'i': 11044, 'not': 8141, 'you': 6354, 'have': 5146, 'no': 3620, 'one': 3498, 'like': 2253, 'more': 2087, 'out': 2021, 'up': 1831, 'man': 1783, 'now': 1579, 'only': 1555, 'must': 1523, 'little': 1485, 'those': 1447, 'good': 1444, 'should': 1417, 'after': 1379, 'great': 1358, 'every': 1356, 'first': 1318, 'own': 1289, 'did': 1271, 'how': 1266, 'see': 1251, 'these': 1244, 'men': 1233, 'over': 1209, 'where': 1205, 'make': 1196, 'upon': 1188, 'nor': 1181, 'never': 1177, 'much': 1167, 'time': 1166, 'said,': 1163, 'two': 1142, 'old': 1140, 'made': 1128}


In [23]:
write_json(top_40_words_without_stop_words_dictionary,"sp2.json")

In [24]:
sc.stop()